In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Thu Oct 31 08:44:36 PDT 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.971
BogoMIPS:              5851.97
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables
`%env` variables are good for passing to bash cells

In [2]:
# Set workding directory
%env wd=/home/sam/analyses/20191031_pgen_v074_stringtie_BAM_splitting
wd="/home/sam/analyses/20191031_pgen_v074_stringtie_BAM_splitting"

%env rsync_gannet=gannet:/volume2/web/Atumefaciens/20190723_stringtie_pgen_v074/
%env wget_bam=--quiet --no-check-certificate https://gannet.fish.washington.edu/Atumefaciens/20190723_stringtie_pgen_v074/20190723_sorted.merged.bam
%env original_bam=20190723_sorted.merged.bam
%env reassembled_bam=20190723_sorted.merged.reassembled.bam

env: wd=/home/sam/analyses/20191031_pgen_v074_stringtie_BAM_splitting
env: rsync_gannet=gannet:/volume2/web/Atumefaciens/20190723_stringtie_pgen_v074/
env: wget_bam=--quiet --no-check-certificate https://gannet.fish.washington.edu/Atumefaciens/20190723_stringtie_pgen_v074/20190723_sorted.merged.bam
env: original_bam=20190723_sorted.merged.bam
env: reassembled_bam=20190723_sorted.merged.reassembled.bam


#### Create necessary directories

In [3]:
%%bash
mkdir --parents ${wd}

In [4]:
cd {wd}

/home/sam/analyses/20191031_pgen_v074_stringtie_BAM_splitting


#### Download Pgen_v074 merged Stringtie BAM file

Info on the BAM is here: https://robertslab.github.io/sams-notebook/2019/07/23/Genome-Annotation-Pgenerosa_v074-Transcript-Isoform-ID-with-Stringtie-on-Mox.html

#### If need to download via wget, uncomment lines in the cell below

In [5]:
# %%bash
# time \
# wget "${wget_gffs}"
# wget "${wget_fasta}"
# ls -lh ${wd}

In [6]:
%%bash
rsync \
--archive \
--verbose \
"${rsync_gannet}${original_bam}" \
.

receiving incremental file list
20190723_sorted.merged.bam

sent 30 bytes  received 77,940,299,973 bytes  64,869,163.55 bytes/sec
total size is 77,930,786,826  speedup is 1.00


In [7]:
%%bash
ls -lh

total 73G
-rw-rw-r-- 1 sam users 73G Aug 29 10:54 20190723_sorted.merged.bam


### Split BAM into 5GB chunks

In [ ]:
%%bash
split \
--bytes 5GB
"${original_bam}" \
"${orignal_bam}_"

ls -lh

### Cleanup

In [25]:
%%bash
rm "${genome_fasta}"* "${exon_comp_bed}" *.gff3 *.txt
ls -ltrh

total 12M
-rw-rw-r-- 1 sam sam 2.1M Oct 30 08:41 Panopea-generosa-vv0.74.a4.intergenic.bed
-rw-rw-r-- 1 sam sam 9.1M Oct 30 08:41 Panopea-generosa-vv0.74.a4.introns.bed


rm: cannot remove 'Pgenerosa_v074.fa*': No such file or directory
rm: cannot remove 'Panopea-generosa-vv0.74.a4.exon.sorted.comp.bed': No such file or directory
